This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'DkWwRyzzR_syzjL3mUjA'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json

In [3]:
# Calling the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json"

#Setting parameters of url for the request 
param = {'api_key':API_KEY,'limit':1}

#Sending get request with the api url and parameters
res = requests.get(url, params=param)

#Stroing object in dictionary: row_dict
row_dict = res.json()

#print(row_dict)
data = json.loads(res.text)
print(json.dumps(data, indent=2))

{
  "dataset": {
    "id": 10095370,
    "dataset_code": "AFX_X",
    "database_code": "FSE",
    "name": "Carl Zeiss Meditec (AFX_X)",
    "description": "Stock Prices for Carl Zeiss Meditec (2020-05-15) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704",
    "refreshed_at": "2020-05-29T22:16:21.973Z",
    "newest_available_date": "2020-05-29",
    "oldest_available_date": "2000-06-07",
    "column_names": [
      "Date",
      "Open",
      "High",
      "Low",
      "Close",
      "Change",
      "Traded Volume",
      "Turnover",
      "Last Price of the Day",
      "Daily Traded Units",
      "Daily Turnover"
    ],
    "frequency": "daily",
    "type": "Time Series",
    "premium": false,
    "limit": 1,
    "transform": null,
    "column_index": null,
    "start_date": "2000-06-07",
    "end_date": "2020-05-29",
    "data": [
      [
        "2020-05-29",
        91.05,
        93.1,
        90.25,
        92.1,
        null,
        23816

In [4]:
# Inspecting the JSON structure of the object created, and taking note of all nested dictionaries

#function to find nested dictionaries
def findKeys(temp_dict,dstring):
    keys_n = temp_dict.keys()
    if(len(keys_n)>0):
        dstring+='{' 
        for key in keys_n:
            dstring += key + ': ,\n'
            #print(key)
            if isinstance(temp_dict[key], dict):     
                temp_dict = temp_dict[key]
                dstring+='\t'+findKeys(temp_dict,'') + '\n'
        dstring+='}'
    return(dstring)

#checking for nested dictionaries in the JSON dictionary object recieved
keys = row_dict.keys()
dict_string = findKeys(row_dict, '')
print(dict_string)

{dataset: ,
	{id: ,
dataset_code: ,
database_code: ,
name: ,
description: ,
refreshed_at: ,
newest_available_date: ,
oldest_available_date: ,
column_names: ,
frequency: ,
type: ,
premium: ,
limit: ,
transform: ,
column_index: ,
start_date: ,
end_date: ,
data: ,
collapse: ,
order: ,
database_id: ,
}
}


In [5]:
#Inspecting the type of column_names and data in JSON to check for consistency

#checking the type of values in dictionary columns_names
print(type(row_dict['dataset']['column_names']))

#checking the type of values in dictionary data which has the data for these columns
print(type(row_dict['dataset']['data']))

<class 'list'>
<class 'list'>


In [6]:
#printing the list of columns names and their indexes for further data analysis

lst= row_dict['dataset']['column_names']

print([{x,lst.index(x)} for x in lst])

[{0, 'Date'}, {'Open', 1}, {2, 'High'}, {'Low', 3}, {4, 'Close'}, {'Change', 5}, {'Traded Volume', 6}, {'Turnover', 7}, {'Last Price of the Day', 8}, {'Daily Traded Units', 9}, {10, 'Daily Turnover'}]


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [7]:
#1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017

#Adding date range for whole year 2017
param.update({'start_date':'2017-01-01','end_date':'2017-12-31'})

#Deleting the 'limit' parameter
del param['limit']

#Requesting data using the API get request
res = requests.get(url, params=param)

#checking the status of response
print(res.status_code)

200


In [8]:
#2. Converting the JSON object into a Python dictionary: fse_data
fse_dict = res.json()

In [9]:
#extracting the list of values from nested dictionary 'data' in dictionary 'dataset'
d_list = fse_dict['dataset']['data']

#Checking if there are None values in the nested list inside the data_list
validation = any(None in sl for sl in d_list)
print("Data has None values: "+ str(validation))

Data has None values: True


DATA IMPUTATION CAN BE DONE CONVENIENTLY USING PANDAS DATAFRAME, SO I HAVE CHOSEN TO CONVERT DATASET INTO A DATAFRAME, HOWEVER AFTER IMPUTATION I WILL CONVERT THE DATA BACK TO LIST TO PERFORM THE REMAINING STEPS

In [10]:
# Usig pandas Dataframe for Imputing Data -- I will be filling the None values in stock prices data with values from the previous days
import pandas as pd

df = pd.DataFrame(d_list,columns=['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'])
print(df.shape)

(255, 11)


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   255 non-null    object 
 1   Open                   252 non-null    float64
 2   High                   255 non-null    float64
 3   Low                    255 non-null    float64
 4   Close                  255 non-null    float64
 5   Change                 1 non-null      float64
 6   Traded Volume          255 non-null    float64
 7   Turnover               255 non-null    float64
 8   Last Price of the Day  0 non-null      object 
 9   Daily Traded Units     0 non-null      object 
 10  Daily Turnover         0 non-null      object 
dtypes: float64(7), object(4)
memory usage: 22.0+ KB


In [12]:
#Imputing only those columns that will be required for further analysis
#1) forward filling 'Open' column, 2) filling 'Change' column with difference between 'High' and 'Low' Column values  
df['Open'].fillna(method='ffill',inplace=True)
df['Change'].fillna(abs(df['High']-df['Low']),inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   255 non-null    object 
 1   Open                   255 non-null    float64
 2   High                   255 non-null    float64
 3   Low                    255 non-null    float64
 4   Close                  255 non-null    float64
 5   Change                 255 non-null    float64
 6   Traded Volume          255 non-null    float64
 7   Turnover               255 non-null    float64
 8   Last Price of the Day  0 non-null      object 
 9   Daily Traded Units     0 non-null      object 
 10  Daily Turnover         0 non-null      object 
dtypes: float64(7), object(4)
memory usage: 22.0+ KB


In [13]:
#Checking for duplicate data 
duplicateRowsDF = df[df.duplicated()]
print(duplicateRowsDF)

Empty DataFrame
Columns: [Date, Open, High, Low, Close, Change, Traded Volume, Turnover, Last Price of the Day, Daily Traded Units, Daily Turnover]
Index: []


There are no duplicate rows found in the dataframe so nothing needs to be done for duplicates.

In [14]:
#converting dataframe back to list for further steps
data_list = df.values.tolist()

In [15]:
#3. Calculating the highest and lowest opening prices were for the stock in this period. 
#  'Open' prices is the 2nd item in the list of columns values and therefore the 2nd list item in the data_list as well 

#initializing highest open price as 0: max_open
max_open=0

#initializing lowest open price as the first value in the list: min_open
min_open=data_list[0][1]

#finding maximum and minimum open price from the list of lists
for lst in data_list:
    max_open = max(max_open,lst[1])
    min_open = min(min_open,lst[1])

#priting the maximum and minimum open price
print("Higest opening prices during the year 2017: $" + str(max_open))
print("Lowest opening prices during the year 2017: $" + str(min_open))

Higest opening prices during the year 2017: $53.11
Lowest opening prices during the year 2017: $34.0


In [16]:
#4. Calculating the largest change in any one day (based on High and Low price)

#initializing largest_change variable
largest_change=abs(data_list[0][2]-data_list[0][3])

#finding largest change in any one day
for lst in data_list:
    largest_change = max(largest_change,abs(lst[2]-lst[3]))
    
#printing the largest change in any one day
print("Largest change in any one day during 2017: $" +str(largest_change))

Largest change in any one day during 2017: $2.8100000000000023


In [17]:
#5. Calculating the largest change between any two days (based on Closing Price)?

max_close=data_list[0][4]
min_close=data_list[0][4]

#finding maximum and minimum closing price from the list of lists
for lst in data_list:
    max_close = max(max_open,lst[4])
    min_close = min(min_open,lst[4])
        
#finding the difference between largest and smallest closing prices
max_close_diff = max_close-min_close

#printing the maximum closing price difference
print("Largest change in any two days based on Closing Price during 2017: $" + str(max_close_diff))

Largest change in any two days based on Closing Price during 2017: $19.11


In [18]:
#6. Calculating the average daily trading volume during this year?

#initializing variable for total colume recored during 2017: total_vol
total_vol=0

#initializing count of days recorded during 2017: day_count
day_count=len(data_list)

#finding total volume during the year
for lst in data_list:
    total_vol += lst[6]

#finding average volume: avg_vol
avg_vol = total_vol/day_count   

#printing the average volume
print("Average daily trading volume during 2017: $" + str(avg_vol))

Average daily trading volume during 2017: $89124.33725490196


In [19]:
#Calculating the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#Making an empty list of trading volume during this year: list_vol
list_vol=[]

#Inserting trading volumes by each day during the year 2017
for lst in data_list:
    list_vol.append(lst[6])

#function to calculate the median of a list of trading volumes
def calculate_median(lst):
    l = sorted(lst)
    l_len = len(lst)
    if l_len < 1:
        return None
    if l_len % 2 == 0 :
        return (round( l[round((l_len-1)/2)] + l[round((l_len+1)/2)] ) / 2.0)
    else:
        return l[round((l_len-1)/2)]

#printing the median trading volume this year
print("The median trading volume during this year is: $"+ str(calculate_median(list_vol)))

The median trading volume during this year is: $76286.0
